In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from gurobipy import *
from BOM_graph.StudyBOM import GenerateGraph

In [2]:
# Definicion de los parametros del entorno gurobi

#Dado el número de variables de este modelo, es necesario utilizar gurobi con una licencia académica (se obtiene gratuitamente en la web de gurobi)
params = {"WLSACCESSID" : '5cbfde7e-b7bd-44e3-9e2d-fb335f5a2deb', "WLSSECRET" : 'f526471f-ed2c-462d-a933-1241228cd704', "LICENSEID" : 2489204 }
env = Env(params = params)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2489204
Academic license 2489204 - for non-commercial use only - registered to 80___@unizar.es


In [11]:
# Definición de los parámetros del problema
BOM = pd.read_pickle('./DataFiles/BOM.pkl')
BOM = BOM[BOM['MyBOMITEMID'] != 44].reset_index(drop=True) # Delete non connected element
MixedItems = pd.read_pickle('./DataFiles/MixedItems.pkl')
MixedItems = MixedItems[MixedItems['MyBOMITEMID'] != 44].reset_index(drop=True) # Delete non connected element
PurchaseItems = pd.read_pickle('./DataFiles/PurchaseItems.pkl')
RouteItems = pd.read_pickle('./DataFiles/RouteItems.pkl')
Orders = pd.read_pickle('./DataFiles/Orders.pkl')
Orders = Orders[Orders['MyBOMITEMID'] != 44].reset_index(drop=True)
Orders['END_DATE'] = pd.to_datetime(Orders['END_DATE'])


# Grafo
G = GenerateGraph(BOM, typeG_ND = False, connected = True)

# Items
NN = sorted(G.nodes)
NN_len = sorted(NN)
K1 = sorted(RouteItems["MyBOMITEMID"].unique().tolist())
K2 = sorted(PurchaseItems["MyBOMITEMID"].unique().tolist())
K3 = sorted(MixedItems["MyBOMITEMID"].unique().tolist())

# Arcos 
arcos = sorted(G.edges)

# Layers
layers_dict = {}
for node in G.nodes:
    layer = G.nodes[node]["layer"]
    if layer not in layers_dict:
        layers_dict[layer] = []
    layers_dict[layer].append(node)
layers = [sorted(nodes) for layer, nodes in sorted(layers_dict.items())]

# Conjuntos N(i)
N = {}
# Encontrar los ítems necesarios para cada ítem i en K1 ∪ K3
for i in sorted(K1+K3):
    N[i] = [j for _, j in G.edges(i)]

# Clientes
R = sorted(Orders["CUSTOMERID"].unique().tolist())
R_len = len(R)

# Periodos de tiempo en el horizonte temporal
date_range = pd.date_range(start= dt.datetime.now().strftime("%Y-%m-01"), periods=13, freq='MS')
T = date_range.tolist() # De hoy a 12 meses

# Demanda y precios de venta
# Map MyBOMITEMID and CUSTOMERID to indices
item_indices = {item: idx for idx, item in enumerate(layers[0]+[39])} # El 39 es un ítem a nivel 0 y Layer 1
customer_indices = {customer: idx for idx, customer in enumerate(R)}
D = []
B = []
for period_start in T:
    period_end = period_start + pd.DateOffset(months=1) - pd.DateOffset(days=1)  # End of the month
    period_df = Orders[(Orders['END_DATE'] >= period_start) & (Orders['END_DATE'] <= period_end)]
    
    # Create matrices for quantities and prices
    period_matrixD = np.zeros((len(layers[0]+[39]), R_len))
    period_matrixB = np.zeros((len(layers[0]+[39]), R_len))
    
    # Fill the matrices
    for _, row in period_df.iterrows():
        item_idx = item_indices[row['MyBOMITEMID']]
        customer_idx = customer_indices[row['CUSTOMERID']]
        period_matrixD[item_idx, customer_idx] = row['QUANTITY']
        period_matrixB[item_idx, customer_idx] = row['UNITPRICE_EUR']
    
    # Append the period matrices to the lists
    D.append(period_matrixD)
    B.append(period_matrixB)
    
    # Costes 
    c1 = {**dict(zip(RouteItems["MyBOMITEMID"], RouteItems["RUNTIME_COST"])), 
      **dict(zip(MixedItems["MyBOMITEMID"], MixedItems["RUNTIME_COST"]))}
    c2 = {**dict(zip(PurchaseItems["MyBOMITEMID"], PurchaseItems["UNITPRICE_Compra"])), 
      **dict(zip(MixedItems["MyBOMITEMID"], MixedItems["UNITPRICE_Compra"]))}
    
    # MOQs
    MOQ1 = {**dict(zip(RouteItems["MyBOMITEMID"], RouteItems["MOQ_Fabricacion"])), 
      **dict(zip(MixedItems["MyBOMITEMID"], MixedItems["MOQ_Fabricacion"]))}
    MOQ2 = {**dict(zip(PurchaseItems["MyBOMITEMID"], PurchaseItems["MOQ_Compra"])), 
      **dict(zip(MixedItems["MyBOMITEMID"], MixedItems["MOQ_Compra"]))}
    
    # Lead times
    lt = {**dict(zip(PurchaseItems["MyBOMITEMID"], PurchaseItems["LEADTIME"])), 
      **dict(zip(MixedItems["MyBOMITEMID"], MixedItems["LEADTIME"]))}
    
    # Stock
    
    # Matriz alpha
    alpha = {}
    for i in sorted(K1 + K3):
        alpha[i] = {}
        for j in N[i]:
            alpha[i][j] = BOM[(BOM['MyBOMITEMID'] == i) & (BOM['MyPARENTBOMITEMID'] == j)]['MAXIBOQTY'].values[0]
            
    print('a')

KeyError: 'MAXIBOQTY'